# No Glue Code

In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/Cambdrige`


In [2]:
using Random
using LinearAlgebra
using PyPlot

#What we are tweaking
using Revise
using Turing
using MCMCChains

[ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]
┌ Warning: Module ForwardDiffStaticArraysExt with build ID fafbfcfd-1e5e-f6ad-0000-175e66c2c915 is missing from the cache.
│ This may mean ForwardDiffStaticArraysExt [b74fd6d0-9da7-541f-a07d-1b6af30a262f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1758
┌ Error: Error during loading of extension ForwardDiffStaticArraysExt of ForwardDiff, use `Base.retry_load_extensions()` to retry.
│   exception =
│    1-element ExceptionStack:
│    Declaring __precompile__(false) is not allowed in files that are being precompiled.
│    Stacktrace:
│      [1] _require(pkg::Base.PkgId, env::Nothing)
│        @ Base ./loading.jl:1762
│      [2] _require_prelocked(uuidkey::Base.PkgId, env::Nothing)
│        @ Base ./loading.jl:1625
│      [3] _require_prelocked(uuidkey::Base.PkgId)
│        @ Base ./loading.jl:1623
│      [4] run_extension_callbacks(extid::Base.ExtensionId)
│        @ Base ./l

## Model

In [3]:
# Just a simple Neal Funnel
d = 21
@model function funnel()
    θ ~ Normal(0, 3)
    z ~ MvNormal(zeros(d-1), exp(θ)*I)
    x ~ MvNormal(z, I)
end

funnel (generic function with 2 methods)

In [4]:
Random.seed!(1)
(;x) = rand(funnel() | (θ=0,))
funnel_model = funnel() | (;x)

Model{typeof(funnel), (), (), (), Tuple{}, Tuple{}, ConditionContext{NamedTuple{(:x,), Tuple{Vector{Float64}}}, DefaultContext}}(funnel, NamedTuple(), NamedTuple(), ConditionContext((x = [1.2142074831535152, 1.23371919965455, -0.8480146960461767, 0.1600994648479841, 1.9180385508479283, -3.401523464506408, -0.0957684186471088, 0.6734622629464286, -3.2749467689509633, -1.6760091758453226, 1.9567202902549736, 0.1136169088905351, 0.11117896909388916, -0.5373922347882832, -0.12436857036298687, -1.2901071061088532, 1.702584517514787, -0.44460133117954226, 1.0818722439221686, 1.2208011493237483],), DefaultContext()))

## Sampling

In [5]:
nadapts=500 
TAP=0.95
nuts = NUTS(nadapts, TAP; init_ϵ=0.1)

NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}(500, 0.95, 10, 1000.0, 0.1)

In [6]:
nuts_samples = sample(funnel_model, nuts, 5000)

LoadError: MethodError: no method matching sample(::Model{typeof(funnel), (), (), (), Tuple{}, Tuple{}, ConditionContext{NamedTuple{(:x,), Tuple{Vector{Float64}}}, DefaultContext}}, ::NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}, ::Int64)

[0mClosest candidates are:
[0m  sample(::Model, [91m::AbstractMCMC.AbstractSampler[39m, ::Integer; progress, verbose, callback, kwargs...)
[0m[90m   @[39m [35mTuring[39m [90m~/Cambdrige/Turing.jl/src/inference/[39m[90m[4mInference.jl:249[24m[39m
[0m  sample(::Any, [91m::AbstractMCMC.AbstractSampler[39m, ::Any; kwargs...)
[0m[90m   @[39m [32mAbstractMCMC[39m [90m~/.julia/packages/AbstractMCMC/bE6VB/src/[39m[90m[4msample.jl:15[24m[39m
[0m  sample([91m::AbstractRNG[39m, [91m::Chains[39m, ::Integer; replace, ordered)
[0m[90m   @[39m [35mMCMCChains[39m [90m~/.julia/packages/MCMCChains/OVsxE/src/[39m[90m[4msampling.jl:26[24m[39m
[0m  ...
